In [1]:
#!/usr/bin/env python

# ROSpy node to subscribe to a ROS image topic
# Taken and modified from http://wiki.ros.org/rospy_tutorials/Tutorials/WritingImagePublisherSubscriber

from __future__ import division
#from __future__ import print_function

# Python libs
import sys, time

# numpy and scipy
import numpy as np
from scipy.ndimage import filters

from features_extraction import points_extraction

# OpenCV
import cv2

# Ros libraries
import roslib
import rospy

# Ros Messages
from sensor_msgs.msg import CompressedImage
# We do not use cv_bridge it does not support CompressedImage in python
# from cv_bridge import CvBridge, CvBridgeError

VERBOSE=False

class image_processing:

    def __init__(self):
        
        '''Initialize ros publisher, ros subscriber'''
        # topic where we publish
        #self.image_pub = rospy.Publisher("/output/image_raw/compressed",
        #    CompressedImage)
        #self.bridge = CvBridge()
            
        # subscribed Topic
        self.subscriber = rospy.Subscriber("/camera/rgb/image_raw/compressed",
            CompressedImage, self.callback,  queue_size = 1)
        if VERBOSE :
            print "subscribed to /camera/rgb/image_raw/compressed"
            
        # Instantiate the point extraction class
        self.points_ex = points_extraction()
    
    def callback(self, ros_data):

        '''Callback function of subscribed topic.'''
        if VERBOSE :
            print 'received image of type: "%s"' % ros_data.format
                
        #### direct conversion to CV2 ####
        np_arr = np.fromstring(ros_data.data, np.uint8)

        #image_np = cv2.imdecode(np_arr, cv2.CV_LOAD_IMAGE_COLOR)
        image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR) # OpenCV >= 3.0:
        
        # Feature extraction
        self.points_ex.run(image)
        
        # Get the processed image
        image_proc = self.points_ex.get_image()
    
        cv2.imshow('Robot camera', image_proc)
        
        key = cv2.waitKey(1)

        self.stop_it = False
        if key == ord('q'):
            self.stop_it = True
            cv2.destroyAllWindows()
            rospy.signal_shutdown('Quit')
        
        #### Create CompressedIamge ####
        #msg = CompressedImage()
        #msg.header.stamp = rospy.Time.now()
        #msg.format = "jpeg"
        #msg.data = np.array(cv2.imencode('.jpg', image_np)[1]).tostring()
        # Publish new image
        #self.image_pub.publish(msg)
        
        #self.subscriber.unregister()

def main(args):

    '''Initializes and cleanup ros node'''
    
    ic = image_processing()
    
    rospy.init_node('image_processing', anonymous=True)
    
    try:
        rospy.spin()
    except KeyboardInterrupt:
        print "Shutting down ROS image_processing module"
    
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main(sys.argv)

/home/toto/miniconda2/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
